In [1]:
import sys
import os
import time

import numpy as np
import pandas as pd
import sklearn
import requests

from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 16})

# Let's check our software versions
print('------------')
print('### Python version: ' + __import__('sys').version)
print('### NumPy version: ' + np.__version__)
print('### Scikit-learn version: ' + sklearn.__version__)
print('------------')

def var_exists(var_name):
    return (var_name in globals() or var_name in locals())

------------
### Python version: 3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ]
### NumPy version: 1.26.0
### Scikit-learn version: 1.2.1
------------


In [2]:
# Get tech tickers
r = requests.get('https://swamplocksapi.azurewebsites.net/api/Financials/stocks')
r.status_code
df = pd.DataFrame(r.json())

techTickers = []
for i in range(463):
    techTickers.append(df.iloc[i][0])
    # if df.iloc[i][1] == 'Information Technology':
    #     techTickers.append(df.iloc[i][0])

In [3]:
# AAPL specific prediction from here down.
# Load AAPL's data

# AAPL closing price stored in dfAAPLclose
r = requests.get('https://swamplocksapi.azurewebsites.net/api/Financials/stocks/AAPL/filtered_data')
dfAAPLclose = pd.DataFrame(r.json())
dfAAPLclose['date'] = pd.to_datetime(dfAAPLclose['date'])
dfAAPLclose = dfAAPLclose.sort_values('date')

# 'dfbs' stores AAPL's data from the balance sheet
aaplbsURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/balancesheets/AAPL'
r = requests.get(aaplbsURL)
dfbs = pd.DataFrame(r.json())

# 'dfcf' stores AAPL's data from cash flow statement
aaplcfURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/cashflowstatements/AAPL'
r = requests.get(aaplcfURL)
dfcf = pd.DataFrame(r.json())

# 'dfe' stores AAPL's data from earnings statement
aapleURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/earnings/AAPL'
r = requests.get(aapleURL)
dfe = pd.DataFrame(r.json())

# 'dfis' stores AAPL's data from income statement
aaplisURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/incomestatements/AAPL'
r = requests.get(aaplisURL)
dfis = pd.DataFrame(r.json())

In [4]:
df_closing_price = dfAAPLclose[['date', 'closingPrice']]

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
# Ensure datetime consistency
dfe['fiscalDateEnding'] = pd.to_datetime(dfe['fiscalDateEnding'])
dfcf['fiscalDateEnding'] = pd.to_datetime(dfcf['fiscalDateEnding'])
dfis['fiscalDateEnding'] = pd.to_datetime(dfis['fiscalDateEnding'])
df_closing_price['date'] = pd.to_datetime(df_closing_price['date'])

# Create next quarter end date
dfe['nextQuarterEnd'] = dfe['fiscalDateEnding'] + pd.DateOffset(months=3)
dfcf['nextQuarterEnd'] = dfcf['fiscalDateEnding'] + pd.DateOffset(months=3)
dfis['nextQuarterEnd'] = dfis['fiscalDateEnding'] + pd.DateOffset(months=3)

# Sort by date
dfe = dfe.sort_values('nextQuarterEnd')
dfcf = dfcf.sort_values('nextQuarterEnd')
dfis = dfis.sort_values('nextQuarterEnd')
df_closing_price = df_closing_price.sort_values('date')

merged_financials = pd.merge(
    dfe[['ticker', 'fiscalDateEnding', 'nextQuarterEnd', 'reportedEPS']],
    dfcf[['ticker', 'fiscalDateEnding', 'netIncome', 'paymentsForRepurchaseOfCommonStock', 'profitLoss']],
    on=['ticker', 'fiscalDateEnding'],
    how='inner'
)

# now merge the result with dfis
merged_financials = pd.merge(
    merged_financials,
    dfis[['ticker', 'fiscalDateEnding', 'researchAndDevelopment', 'operatingExpenses', 'sellingGeneralAndAdministrative']],
    on=['ticker', 'fiscalDateEnding'],
    how='inner'
)

# Merge with closing price using merge_asof
final_data = pd.merge_asof(
    merged_financials,
    df_closing_price,
    left_on='nextQuarterEnd',
    right_on='date',
    direction='forward',
    tolerance=pd.Timedelta("30 days")
)

# Filter out rows without marketCap
final_data = final_data.dropna(subset=['closingPrice']).reset_index(drop=True)
"""
researchAndDevelopment               0.953399
operatingExpenses                    0.935023
sellingGeneralAndAdministrative      0.891001
"""
# Define features (X) and target (y)  cashFlowFromFinancing paymentsForRepurchaseOfEquity
features = ['reportedEPS', 'netIncome', 'paymentsForRepurchaseOfCommonStock', 
            'profitLoss',
            'researchAndDevelopment', 'operatingExpenses', 'sellingGeneralAndAdministrative']
X = final_data[features]
y = final_data['closingPrice']

# Verify shapes
print("X shape:", X.shape)  #
print("y shape:", y.shape)  # 

# Split data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Display shapes
print(f'Training set: {X_train.shape}, {y_train.shape}')  # (34, 5), (34,)
print(f'Validation set: {X_val.shape}, {y_val.shape}')  # (11, 5), (11,)
print(f'Test set: {X_test.shape}, {y_test.shape}')      # (12, 5), (12,)

X shape: (62, 7)
y shape: (62,)
Training set: (37, 7), (37,)
Validation set: (12, 7), (12,)
Test set: (13, 7), (13,)


/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/2899205226.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_closing_price['date'] = pd.to_datetime(df_closing_price['date'])


In [6]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import joblib

# Log-transform y
y_log = np.log1p(y)

# Split data
X_train, X_temp, y_train_log, y_temp_log = train_test_split(X, y_log, test_size=0.4, random_state=42)
X_val, X_test, y_val_log, y_test_log = train_test_split(X_temp, y_temp_log, test_size=0.5, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Grid search for SVR
param_grid = {
    'C': [10, 100, 1000],
    'gamma': [0.01, 0.1, 1],
    'kernel': ['rbf']
}
svm = SVR()
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train_log)

# Best model
best_svm = grid_search.best_estimator_
print("Best params:", grid_search.best_params_)

# Predict and evaluate on log scale
y_val_pred_log = best_svm.predict(X_val_scaled)
val_mse_log = mean_squared_error(y_val_log, y_val_pred_log)
print(f"Validation MSE (log scale): {val_mse_log}")

y_test_pred_log = best_svm.predict(X_test_scaled)
test_mse_log = mean_squared_error(y_test_log, y_test_pred_log)
print(f"Test MSE (log scale): {test_mse_log}")

# Convert back to original scale
y_val_pred = np.expm1(y_val_pred_log)
y_test_pred = np.expm1(y_test_pred_log)
val_mse = mean_squared_error(np.expm1(y_val_log), y_val_pred)
test_mse = mean_squared_error(np.expm1(y_test_log), y_test_pred)
print(f"Validation MSE (original scale): {val_mse}")
print(f"Test MSE (original scale): {test_mse}")

ticker = "AAPL"

model_dir = os.path.join('Models', ticker)
os.makedirs(model_dir, exist_ok=True)

# save model
joblib.dump(best_svm, os.path.join(model_dir, 'best_svm_model.pkl'))

# save scalar
joblib.dump(scaler, os.path.join(model_dir, 'scaler.pkl'))

# save input
latest_data = X.iloc[-1].to_frame().T
future_quarters = 1
future_X = pd.DataFrame([latest_data.values[0]] * future_quarters, columns=features)

joblib.dump(future_X, os.path.join(model_dir, f'{ticker}_latest_features.pkl'))

Best params: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Validation MSE (log scale): 0.015095776818325586
Test MSE (log scale): 0.03987756655749405
Validation MSE (original scale): 191.21461161683396
Test MSE (original scale): 524.6287594191847


['Models/AAPL/AAPL_latest_features.pkl']

In [7]:

# Latest data for prediction (last row of X)
latest_data = X.iloc[-1].to_frame().T

# Scale and predict for next 2 quarters
future_quarters = 1
future_X = pd.DataFrame([latest_data.values[0]] * future_quarters, columns=features)
future_X_scaled = scaler.transform(future_X)
future_log_preds = best_svm.predict(future_X_scaled)
future_cp_preds = np.expm1(future_log_preds)

In [8]:
# Predicted price, last quarter's price
lastQuartersPrice = y.iloc[-1]
predictedQuartersPrice = future_cp_preds[0]
print('Projected price next quarter:' , predictedQuartersPrice , 'Last quarter\'s price:' , lastQuartersPrice)
print('Projected percent change by next quarter:' , (1 - lastQuartersPrice/predictedQuartersPrice) * 100)

Projected price next quarter: 200.90589140250228 Last quarter's price: 222.13
Projected percent change by next quarter: -10.564204189998861


In [9]:
r = requests.get('https://swamplocksapi.azurewebsites.net/api/Financials/economic_data/indicators')
econIndicatorsDF = pd.DataFrame(r.json())
indicatorNames = []
for i in range(len(econIndicatorsDF)):
    indicatorNames.append(econIndicatorsDF.iloc[i][0])
print(indicatorNames)
econDFs = {}
for name in indicatorNames:
    print(name)
    r = requests.get('https://swamplocksapi.azurewebsites.net/api/Financials/economic_data/' + name)
    try:
        curDF = pd.DataFrame(r.json())
        curDF['date'] = pd.to_datetime(curDF['date'])
        econDFs[name] = curDF.sort_values('date')
        #econDFs[name] = curDF
    except:
        continue
econ_features = ['10Y-TCMR', '2Y-TCMR', '30Y-TCMR', '3M-TCMR', '5Y-TCMR', '7Y-TCMR', 'CPI', 'Durables', 'FFR', 'GDP', 'GDPPC', 'Inflation', 'RetailSales', 'Unemployment']


['10Y-TCMR', '2Y-TCMR', '30Y-TCMR', '3M-TCMR', '5Y-TCMR', '7Y-TCMR', 'ALL_COMMODITIES', 'CPI', 'Durables', 'FFR', 'GDP', 'GDPPC', 'Inflation', 'RetailSales', 'Unemployment']
10Y-TCMR
2Y-TCMR
30Y-TCMR
3M-TCMR
5Y-TCMR
7Y-TCMR
ALL_COMMODITIES
CPI
Durables
FFR
GDP
GDPPC
Inflation
RetailSales
Unemployment


In [11]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
failed = []
results = []
loop = 1
if loop:
    for t in techTickers:
        try:
            print(t)
            # AAPL closing price stored in dfAAPLclose
            r = requests.get('https://swamplocksapi.azurewebsites.net/api/Financials/stocks/' + t + '/filtered_data')
            dfAAPLclose = pd.DataFrame(r.json())
            dfAAPLclose['date'] = pd.to_datetime(dfAAPLclose['date'])
            dfAAPLclose = dfAAPLclose.sort_values('date')

            # 'dfbs' stores AAPL's data from the balance sheet
            aaplbsURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/balancesheets/' + t
            r = requests.get(aaplbsURL)
            dfbs = pd.DataFrame(r.json())

            # 'dfcf' stores AAPL's data from cash flow statement
            aaplcfURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/cashflowstatements/' + t
            r = requests.get(aaplcfURL)
            dfcf = pd.DataFrame(r.json())

            # 'dfe' stores AAPL's data from earnings statement
            aapleURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/earnings/' + t
            r = requests.get(aapleURL)
            dfe = pd.DataFrame(r.json())

            # 'dfis' stores AAPL's data from income statement
            aaplisURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/incomestatements/' + t
            r = requests.get(aaplisURL)
            dfis = pd.DataFrame(r.json())
            df_closing_price = dfAAPLclose[['date', 'closingPrice']]

            # Ensure datetime consistency
            dfe['fiscalDateEnding'] = pd.to_datetime(dfe['fiscalDateEnding'])
            dfcf['fiscalDateEnding'] = pd.to_datetime(dfcf['fiscalDateEnding'])
            dfis['fiscalDateEnding'] = pd.to_datetime(dfis['fiscalDateEnding'])
            df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])

            # Create next quarter end date
            dfe['nextQuarterEnd'] = dfe['fiscalDateEnding'] + pd.DateOffset(months=3)
            dfcf['nextQuarterEnd'] = dfcf['fiscalDateEnding'] + pd.DateOffset(months=3)
            dfis['nextQuarterEnd'] = dfis['fiscalDateEnding'] + pd.DateOffset(months=3)

            # Sort by date
            dfe = dfe.sort_values('nextQuarterEnd')
            dfcf = dfcf.sort_values('nextQuarterEnd')
            dfis = dfis.sort_values('nextQuarterEnd')
            df_closing_price = df_closing_price.sort_values('date')

            merged_financials = pd.merge(
                dfe[['ticker', 'fiscalDateEnding', 'nextQuarterEnd', 'reportedEPS']],
                dfcf[['ticker', 'fiscalDateEnding', 'netIncome', 'paymentsForRepurchaseOfCommonStock', 'profitLoss']],
                on=['ticker', 'fiscalDateEnding'],
                how='inner'
            )

            # now merge the result with dfis
            merged_financials = pd.merge(
                merged_financials,
                dfis[['ticker', 'fiscalDateEnding', 'researchAndDevelopment', 'operatingExpenses', 'sellingGeneralAndAdministrative']],
                on=['ticker', 'fiscalDateEnding'],
                how='inner'
            )

            # Merge with closing price using merge_asof
            final_data = pd.merge_asof(
                merged_financials,
                df_closing_price,
                left_on='nextQuarterEnd',
                right_on='date',
                direction='forward',
                tolerance=pd.Timedelta("30 days")
            )

            for econ_name in econ_features:
                if econ_name in econDFs:
                    econ_df = econDFs[econ_name][['date', 'value']].copy()
                    econ_df = econ_df.rename(columns={'value': econ_name})  # Rename value column to indicator name
                    # Aggregate daily/monthly data to quarterly (use last value in quarter)
                    econ_df['quarter_end'] = econ_df['date'] + pd.offsets.QuarterEnd(0)
                    econ_df = econ_df.groupby('quarter_end').last().reset_index()[['quarter_end', econ_name]]
                    #print(f"{econ_name} econ_df shape: {econ_df.shape}")
                    #print(f"{econ_name} econ_df columns: {econ_df.columns.tolist()}")
                    # Merge with final_data
                    final_data = pd.merge_asof(
                        final_data.sort_values('nextQuarterEnd'),
                        econ_df,
                        left_on='nextQuarterEnd',
                        right_on='quarter_end',
                        direction='backward',  # Take the most recent economic data before nextQuarterEnd
                        tolerance=pd.Timedelta("90 days")
                    )
            existing_econ_features = [f for f in econ_features if f in final_data.columns]
            if existing_econ_features:
                #print(f"Imputing missing values for: {existing_econ_features}")
                # Forward fill first, then fill remaining with mean
                final_data[existing_econ_features] = final_data[existing_econ_features].fillna(method='ffill')
                final_data[existing_econ_features] = final_data[existing_econ_features].fillna(final_data[existing_econ_features].mean())
            else:
                print(f"Warning: No economic features merged for {t}")
                    # Drop rows with missing economic data (optional; adjust based on needs)
            final_data = final_data.dropna(subset=econ_features).reset_index(drop=True)

                    # Define features including economic indicators
            features = [
                'reportedEPS', 'netIncome', 'paymentsForRepurchaseOfCommonStock',
                'profitLoss', 'researchAndDevelopment', 'operatingExpenses',
                'sellingGeneralAndAdministrative'
            ] + econ_features
            X = final_data[features]
            y = final_data['closingPrice']

            # Verify shapes
            #print("X shape:", X.shape)  #
            #print("y shape:", y.shape)  # 

            # Split data
            X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
            X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

            # Display shapes
            # print(f'Training set: {X_train.shape}, {y_train.shape}')  # (34, 5), (34,)
            # print(f'Validation set: {X_val.shape}, {y_val.shape}')  # (11, 5), (11,)
            # print(f'Test set: {X_test.shape}, {y_test.shape}')      # (12, 5), (12,)


            # Log-transform y
            y_log = np.log1p(y)

            # Split data
            X_train, X_temp, y_train_log, y_temp_log = train_test_split(X, y_log, test_size=0.4, random_state=42)
            X_val, X_test, y_val_log, y_test_log = train_test_split(X_temp, y_temp_log, test_size=0.5, random_state=42)

            # Scale features
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_val_scaled = scaler.transform(X_val)
            X_test_scaled = scaler.transform(X_test)

            # Grid search for SVR
            param_grid = {
                'C': [10, 100, 1000],
                'gamma': [0.01, 0.1, 1],
                'kernel': ['rbf']
            }
            svm = SVR()
            grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='neg_mean_squared_error')
            grid_search.fit(X_train_scaled, y_train_log)

            # Best model
            best_svm = grid_search.best_estimator_
            #print("Best params:", grid_search.best_params_)

            # Predict and evaluate on log scale
            y_val_pred_log = best_svm.predict(X_val_scaled)
            val_mse_log = mean_squared_error(y_val_log, y_val_pred_log)
            #print(f"Validation MSE (log scale): {val_mse_log}")

            model_dir = os.path.join('Models', t)
            os.makedirs(model_dir, exist_ok=True)

            # save model
            joblib.dump(best_svm, os.path.join(model_dir, 'best_svm_model.pkl'))

            # save scalar
            joblib.dump(scaler, os.path.join(model_dir, 'scaler.pkl'))

            # save input
            latest_data = X.iloc[-1].to_frame().T
            future_quarters = 1
            future_X = pd.DataFrame([latest_data.values[0]] * future_quarters, columns=features)

            joblib.dump(future_X, os.path.join(model_dir, f'{t}_latest_features.pkl'))

            y_test_pred_log = best_svm.predict(X_test_scaled)
            test_mse_log = mean_squared_error(y_test_log, y_test_pred_log)
            #print(f"Test MSE (log scale): {test_mse_log}")

            # Convert back to original scale
            y_val_pred = np.expm1(y_val_pred_log)
            y_test_pred = np.expm1(y_test_pred_log)
            val_mse = mean_squared_error(np.expm1(y_val_log), y_val_pred)
            test_mse = mean_squared_error(np.expm1(y_test_log), y_test_pred)
            #print(f"Validation MSE (original scale): {val_mse}")
            #print(f"Test MSE (original scale): {test_mse}")
            # Latest data for prediction (last row of X)
            latest_data = X.iloc[-1].to_frame().T

            # Scale and predict for next quarter
            future_quarters = 1
            future_X = pd.DataFrame([latest_data.values[0]] * future_quarters, columns=features)
            future_X_scaled = scaler.transform(future_X)
            future_log_preds = best_svm.predict(future_X_scaled)
            future_cp_preds = np.expm1(future_log_preds)
            lastQuartersPrice = y.iloc[-1]
            predictedQuartersPrice = future_cp_preds[0]
            percentChange = (1 - lastQuartersPrice/predictedQuartersPrice) * 100
            print('Projected price next quarter:' , predictedQuartersPrice , 'Last quarter\'s price:' , lastQuartersPrice)
            print('Projected percent change by next quarter:' , percentChange)
            results.append((t, lastQuartersPrice, predictedQuartersPrice, percentChange, val_mse, test_mse))
        except Exception as e:
            failed.append(t)
            print(f"Stock {t} failed. Error {e}")

print(failed)

A


/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:46: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:94: FutureWarning: Passing 'suffixes' whic

Stock A failed. Error Input y contains NaN.
AAPL


/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:46: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:94: FutureWarning: Passing 'suffixes' whic

Projected price next quarter: 200.92214611314367 Last quarter's price: 222.13
Projected percent change by next quarter: -10.555259485887492
ABBV


...

/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:46: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:94: FutureWarning: Passing 'suffixes' whic

Projected price next quarter: 209.43069673228888 Last quarter's price: 231.51
Projected percent change by next quarter: -10.542534409812255
WMB


/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:46: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:94: FutureWarning: Passing 'suffixes' whic

Stock WMB failed. Error 
All the 45 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/chandlerwiggins/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/chandlerwiggins/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py", line 192, in fit
    X, y = self._validate_data(
  File "/Users/chandlerwiggins/anaconda3/lib/python3.10/site-packages/sklearn/base.py", line 565, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/Users/chandlerwiggins/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py", line 1122, in check_X_y
    y = _check_y(y, m

/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:46: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_63535/3191436829.py:94: FutureWarning: Passing 'suffixes' whic

In [12]:
# results contains tuples with the format: 
# stock name, lastQuartersPrice, predictedQuartersPrice, percentChange, val_mse, test_mse
print(results)

[('AAPL', 222.13, 200.92214611314367, -10.555259485887492), ('ABBV', 176.2, 174.84944136813422, -0.7724123230238078), ('ABNB', 119.46, 132.12969110818273, 9.588829733817573), ('ABT', 112.8, 116.22213031461132, 2.9444739184763424), ('ACGL', 91.89, 83.08873423758696, -10.592610229500398), ('ACN', 348.5, 326.53288717132614, -6.727381434369306), ('ADI', 171.46, 155.06790550115232, -10.570913720586672), ('ADM', 50.07, 54.69371252844811, 8.453828264159535), ('ADP', 305.53, 276.3592458121698, -10.55537479924098), ('ADSK', 311.34, 281.63255614087177, -10.548298913378694), ('AEE', 89.62, 79.65608533209779, -12.508667261717932), ('AEP', 109.27, 98.76415001096349, -10.63731119831466), ('AES', 12.65, 26.13528797338664, 51.598007977255136), ('AFL', 102.97, 93.07256463862024, -10.634106194246673), ('AIG', 86.94, 78.57307913028671, -10.648584683616114), ('AIZ', 212.27, 191.9471837152136, -10.587712667323501), ('AJG', 284.46, 257.2587627931658, -10.573492973183507), ('AKAM', 80.5, 89.07964708091673, 9